In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


# define dataset
df = pd.read_csv('bodyfat_dataset.csv')
df.sample(frac = 1) # shuffle
df.head()

,Density,BodyFat,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,1.0708,12.3,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,1.0853,6.1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,1.0414,25.3,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,1.0751,10.4,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,1.0340,28.7,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [2]:
labels = df.loc[:, ['Density', 'BodyFat']]
labels = np.array(labels)

labels.shape

(252, 2)

In [3]:
# drop the data labels
df = df.drop('Density', axis=1)
df = df.drop('BodyFat', axis=1)
df.head()

,Age,Weight,Height,Neck,Chest,Abdomen,Hip,Thigh,Knee,Ankle,Biceps,Forearm,Wrist
0,23,154.25,67.75,36.2,93.1,85.2,94.5,59.0,37.3,21.9,32.0,27.4,17.1
1,22,173.25,72.25,38.5,93.6,83.0,98.7,58.7,37.3,23.4,30.5,28.9,18.2
2,22,154.00,66.25,34.0,95.8,87.9,99.2,59.6,38.9,24.0,28.8,25.2,16.6
3,26,184.75,72.25,37.4,101.8,86.4,101.2,60.1,37.3,22.8,32.4,29.4,18.2
4,24,184.25,71.25,34.4,97.3,100.0,101.9,63.2,42.2,24.0,32.2,27.7,17.7


In [4]:
# scale data
data = np.array(df)

x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X = x_scaler.fit_transform(data)
Y = y_scaler.fit_transform(labels)

print(X.shape)
print(Y.shape)

(252, 13)
(252, 2)


In [5]:
X_train = X[:200]
Y_train = Y[:200]

X_test = X[-52:]
Y_test = Y[-52:]

print(X_train.shape)
print(Y_train.shape)

print(X_test.shape)
print(Y_test.shape)

(200, 13)
(200, 2)
(52, 13)
(52, 2)


In [6]:
import nnet

model = nnet.Sequential([
    nnet.layers.Dense((13, 10), nnet.activation.Tanh),
    nnet.layers.Dense((10, 8), nnet.activation.Tanh),
    nnet.layers.Dense((8, 2), nnet.activation.Sigmoid)
])

model.fit(X_train, Y_train, 500, nnet.loss.MeanSquaredError, nnet.optimizers.RMSProp(), X_val=X_test, Y_val=Y_test)

Epoch: 1 Loss: 0.12379817426685623 Validation Loss: 0.14050318101811088 
Epoch: 2 Loss: 0.05812748279264974 Validation Loss: 0.07564440975213932 
Epoch: 3 Loss: 0.04703234365176614 Validation Loss: 0.051750534659612685 
Epoch: 4 Loss: 0.04343606508632689 Validation Loss: 0.06760017141525196 
Epoch: 5 Loss: 0.037405316972636195 Validation Loss: 0.04596108964614333 
Epoch: 6 Loss: 0.033634991409882666 Validation Loss: 0.050837606530146066 
Epoch: 7 Loss: 0.030651093428679922 Validation Loss: 0.04174659156052127 
Epoch: 8 Loss: 0.02841130376015488 Validation Loss: 0.04164028927328049 
Epoch: 9 Loss: 0.026479874283395603 Validation Loss: 0.03715432328511255 
Epoch: 10 Loss: 0.024900494138333843 Validation Loss: 0.03572751838205565 
Epoch: 11 Loss: 0.023527024426976278 Validation Loss: 0.03288349669828933 
Epoch: 12 Loss: 0.022422599794466716 Validation Loss: 0.031407369004363035 
Epoch: 13 Loss: 0.021488622974624436 Validation Loss: 0.029552653766768928 
Epoch: 14 Loss: 0.02085687985016898

In [7]:
# test model accuracy using Mean Absolute Error
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)
mean_absolute_error(Y_test, y_pred.T)

0.09546488877566717

In [8]:
# actual testing
def test_prediction(index):
    y_pred = model.predict(np.array([X_test[index]]))
    y_true = np.array([Y_test[index]])
    
    print('Prediction', y_scaler.inverse_transform(y_pred.T))
    print('Actual', y_scaler.inverse_transform(y_true))

In [9]:
test_prediction(18)

Prediction [[ 1.03892827 25.63614434]]
Actual [[ 1.0433 24.5   ]]


In [10]:
test_prediction(19)

Prediction [[ 1.05549184 20.44400959]]
Actual [[ 1.0646 15.    ]]


In [11]:
test_prediction(27)

Prediction [[ 1.0381298  26.21384877]]
Actual [[ 1.0418 25.2   ]]
